# <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and Google Colab</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Sunday, October 17th submitted via Canvas by 11:59 pm</p>

This will be a in-class project done in teams of 2. 

In this Project, we will work with CIFAR10 image dataset. 
The starter code to download the database using keras is given below. 
Test the project on Google Colab running on a CPU, GPU and TPU
 

# In every line of code, please write a comment to briefly explain what that line is doing.
Your grades will be based on your understanding of the code you write! 


# Task 1
Convert the features in a form that can be given as input to tensorflow library/functions

In this task you will perform data augmentation. That is, pre-process the data to make the model more robust. Experiment with data augmentation techniques like rotation, translation, horizontal-flip, scaling, ZCA whitening and histogram equalization. 
You can choose any two or more augmentation technique(s) of your choice. 

# Task 2
Try to build a Neural Network model, train on the features and report the accuracy.
Report your observations on the time taken on CPU and GPU (with and without CuDNN kernel) 



1.   Create a CNN based model with 4 hidden layers with 64, 128, 256 and 512 units in each succesive layer. Use a 5x5 convolution kernel and change as necessary. (Use at least 2 augmentations on your input) 
2.   Create an LSTM based model with 1 LSTM layer with 256 units. 



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import time
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

In [ ]:
batch_size = 64
input_dim = 3
units = 256
output_size = 10

def build_model(allow_cudnn_kernel=True):
  if allow_cudnn_kernel:
    lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
  else:
    lstm_layer =  tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(units),
        input_shape=(None, input_dim))
  model = tf.keras.models.Sequential([
      lstm_layer,
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Dense(output_size, activation='softmax')]
  )
  return model

In [ ]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [ ]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
x_train = x_train.reshape(50000, 1024, 3)
x_train = x_train / 255.0

x_test = x_test.reshape(10000, 1024, 3)
x_test = x_test / 255.0

In [ ]:
x_train.shape

(50000, 1024, 3)

In [ ]:
x_train.shape

(50000, 1024, 3)

In [ ]:

# Check if GPU is being used
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# With CudNN with GPU

In [ ]:
start = time.time()
with tf.device('/device:GPU:0'):
  #Build and complie the model.
  model_gpu = build_model(allow_cudnn_kernel=True)

  model_gpu.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])
  
  model_gpu.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size, steps_per_epoch = 100,
          epochs=10)
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/10
100/100 [==============================] - 43s 360ms/step - loss: 2.2397 - accuracy: 0.1739 - val_loss: 2.2895 - val_accuracy: 0.1571
Epoch 2/10
100/100 [==============================] - 35s 352ms/step - loss: 2.1798 - accuracy: 0.2008 - val_loss: 2.2766 - val_accuracy: 0.1797
Epoch 3/10
100/100 [==============================] - 35s 353ms/step - loss: 2.1798 - accuracy: 0.1978 - val_loss: 2.2628 - val_accuracy: 0.1600
Epoch 4/10
100/100 [==============================] - 35s 353ms/step - loss: 2.1766 - accuracy: 0.2017 - val_loss: 2.2365 - val_accuracy: 0.1868
Epoch 5/10
100/100 [==============================] - 35s 354ms/step - loss: 2.1680 - accuracy: 0.1986 - val_loss: 2.2045 - val_accuracy: 0.1948
Epoch 6/10
100/100 [==============================] - 35s 352ms/step - loss: 2.1678 - accuracy: 0.2113 - val_loss: 2.2103 - val_accuracy: 0.1860
Epoch 7/10
100/100 [==============================] - 35s 351ms/step - loss: 2.1646 - accuracy: 0.2017 - val_loss: 2.1631 - val_ac

In [ ]:
print("GPU with CuDNN LSTM test accuracy")
scores = model_gpu.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# Without CudNN with GPU

In [ ]:
start = time.time()
with tf.device('/device:GPU:0'):
  slow_model = build_model(allow_cudnn_kernel=False)
  slow_model.set_weights(slow_model.get_weights())
  slow_model.compile(loss='sparse_categorical_crossentropy', 
                    optimizer='sgd', 
                    metrics=['accuracy'])
  slow_model.fit(x_train, y_train, 
                validation_data=(x_test, y_test),
                verbose=1, steps_per_epoch = 100,
                batch_size=batch_size,
                epochs=5)
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/5
100/100 [==============================] - 251s 2s/step - loss: 2.2275 - accuracy: 0.1703 - val_loss: 2.2886 - val_accuracy: 0.1803
Epoch 2/5
100/100 [==============================] - 246s 2s/step - loss: 2.1737 - accuracy: 0.2044 - val_loss: 2.2757 - val_accuracy: 0.1765
Epoch 3/5
100/100 [==============================] - 247s 2s/step - loss: 2.1774 - accuracy: 0.1997 - val_loss: 2.2618 - val_accuracy: 0.1751
Epoch 4/5
100/100 [==============================] - 246s 2s/step - loss: 2.1736 - accuracy: 0.2061 - val_loss: 2.2445 - val_accuracy: 0.1802
Epoch 5/5
100/100 [==============================] - 242s 2s/step - loss: 2.1719 - accuracy: 0.2075 - val_loss: 2.2142 - val_accuracy: 0.1819
Training time: 1232.570172071457s


In [ ]:
print("GPU without CuDNN LSTM test accuracy")
scores = slow_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# without CudNN and with CPU

In [ ]:
start = time.time()
slow_model = build_model(allow_cudnn_kernel=False)
slow_model.set_weights(slow_model.get_weights())
slow_model.compile(loss='sparse_categorical_crossentropy', 
                   optimizer='sgd', 
                   metrics=['accuracy'])
slow_model.fit(x_train, y_train, 
               validation_data=(x_test, y_test),
               verbose=1,steps_per_epoch = 100,
               batch_size=batch_size,
               epochs=5) 
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/5
100/100 [==============================] - 533s 5s/step - loss: 2.2272 - accuracy: 0.1747 - val_loss: 2.2876 - val_accuracy: 0.1628
Epoch 2/5
100/100 [==============================] - 530s 5s/step - loss: 2.1866 - accuracy: 0.1977 - val_loss: 2.2766 - val_accuracy: 0.1711
Epoch 3/5
100/100 [==============================] - 529s 5s/step - loss: 2.1728 - accuracy: 0.2042 - val_loss: 2.2611 - val_accuracy: 0.2001
Epoch 4/5
 65/100 [==================>...........] - ETA: 2:04 - loss: 2.1583 - accuracy: 0.2060

In [ ]:
print("CPU without CuDNN LSTM test accuracy")
scores = slow_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])